In [1]:
!pip install squarify

In [0]:
import re
import string
from collections import Counter
import squarify
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import spacy
from spacy.tokenizer import Tokenizer

from bs4 import BeautifulSoup
import html as ihtml

import requests
import sqlite3

In [0]:
def extract_job_title(soup):
    jobs = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            jobs.append(a["title"])
    return(jobs)

In [0]:
def extract_company(soup):
    companies = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        company = div.find_all(name="span", attrs={"class":"company"})
        if len(company) > 0:
            for b in company:
                companies.append(b.text.strip())
        else:
            sec_try = div.find_all(name="span", attrs={"class":"result-linl-source"})
            for span in sec_try:
                companies.append(span.text.strip())
    return(companies)

In [0]:
def extract_location(soup):
    locations = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        try:
            locations.append(div.find(name="span", attrs={"class":"location"}).text)
        except:
            locations.append("None")
    return(locations)

In [0]:
def extract_salary(soup):
    salaries = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        try:
            salaries.append(div.find(name="span", attrs={"class":"salaryText"}).text.replace("\n",""))
        except:
            salaries.append("None")
    return(salaries)

In [0]:
def extract_job_title(soup):
    jobs = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            jobs.append(a["title"])
    return(jobs)

In [0]:
def extract_url(soup):
    urls = []
    for div1 in soup.find_all(name="div", attrs={"class":"row"}):
        for div2 in div1.find_all(name="div", attrs={"class":"title"}):
            for a in div2.find_all(name="a", href=True):
                urls.append(a['href'])
    return(urls)

def extract_desc(urls):
    descs = []
    for url in urls:
        full_url = "https://www.indeed.com" + url
        detail_page = requests.get(full_url)
        detail_soup = BeautifulSoup(detail_page.text, "html.parser")
        
        for div in detail_soup.find_all(name="div", attrs={"id":"jobDescriptionText"}):
            descs.append(div.text)
    return(descs)

In [0]:
def extract_count(soup):
    for div in soup.find_all(name="div", attrs={"id":"searchCountPages"}):
        temp_str = div.text.replace(" ", "")
        temp_count_str = re.search('of(.*)jobs', temp_str)
        count_str = re.sub('[^0-9]','', temp_count_str.group(1))
        count = int(count_str)
    return(count)

In [0]:
def extract_list(title_name, city_name, st_name):
    max_results = 100
    columns = ["city", "job_title", "company", "location", "salary", "description"]

    city_url = "https://www.indeed.com/jobs?q=" + title_name + \
               "&l=" + city_name + "%2C+" + st_name
    page = requests.get(city_url)
    soup = BeautifulSoup(page.text, "html.parser")
    max_counter = extract_count(soup)
    print(max_counter, title_name, city_name)

    job_title_list = []
    company_list = []
    location_list = []
    salary_list = []
    desc_list = []

    for start in range(0, max_results, 10):
        city_url = "https://www.indeed.com/jobs?q=" + title_name + \
                   "&l=" + city_name + "%2C+" + st_name + \
                   "&start=" + str(start)
        page = requests.get(city_url)
        soup = BeautifulSoup(page.text, "html.parser")

        job_title_list.extend(extract_job_title(soup))
        company_list.extend(extract_company(soup))
        location_list.extend(extract_location(soup))
        salary_list.extend(extract_salary(soup))
        add_urls = extract_url(soup)
        desc_list.extend(extract_desc(add_urls))

    return job_title_list, company_list, location_list, salary_list, desc_list

def job_db(title, city, st):   
    j_list, c_list, l_list, s_list, d_list = extract_list(title, city, st)     
    temp_df = pd.DataFrame(list(zip(j_list, c_list, l_list, s_list, d_list)), 
                          columns = ['job_title', 'company', 'location', 'salary', 'description'])
    return temp_df

In [11]:
temp1_df = job_db('data+scientist', 'San+Jose', 'CA')
temp2_df = job_db('data+scientist', 'San+Francisco', 'CA')
temp3_df = job_db('data+scientist', 'Seattle', 'WA')
temp4_df = job_db('data+scientist', 'Washington', 'DC')
temp5_df = job_db('data+scientist', 'New+York', 'NY')
temp6_df = job_db('data+scientist', 'Baltimore', 'MD')
temp7_df = job_db('data+scientist', 'Boulder', 'CO')
temp8_df = job_db('data+scientist', 'San+Diego', 'CA')
temp9_df = job_db('data+scientist', 'Denver', 'CO')
temp10_df = job_db('data+scientist', 'Huntsville', 'AL')
temp11_df = job_db('data+scientist', 'Colorado+Springs', 'CO')
temp12_df = job_db('data+scientist', 'Houston', 'TX')
temp13_df = job_db('data+scientist', 'Trenton', 'NJ')
temp14_df = job_db('data+scientist', 'Dallas', 'TX')
temp15_df = job_db('data+scientist', 'Columbus', 'OH')
temp16_df = job_db('data+scientist', 'Austin', 'TX')
temp17_df = job_db('data+scientist', 'Philadelphia', 'PA')
temp18_df = job_db('data+scientist', 'Durham', 'NC')
temp19_df = job_db('data+scientist', 'Raleigh', 'NC')
temp20_df = job_db('data+scientist', 'Atlanta', 'GA')

data_scientist_df = pd.concat([temp1_df, temp2_df, temp3_df, temp4_df, temp5_df,
                       temp6_df, temp7_df, temp8_df, temp9_df, temp10_df,
                       temp11_df, temp12_df, temp13_df, temp14_df, temp15_df,
                       temp16_df, temp17_df, temp18_df, temp19_df, temp20_df], ignore_index=True)

print(data_scientist_df.shape)

1315 data+scientist San+Jose
935 data+scientist San+Francisco
1107 data+scientist Seattle
1176 data+scientist Washington
991 data+scientist New+York
166 data+scientist Baltimore
161 data+scientist Boulder
163 data+scientist San+Diego
180 data+scientist Denver
22 data+scientist Huntsville
6 data+scientist Colorado+Springs
100 data+scientist Houston
92 data+scientist Trenton
258 data+scientist Dallas
74 data+scientist Columbus
185 data+scientist Austin
214 data+scientist Philadelphia
146 data+scientist Durham
148 data+scientist Raleigh
259 data+scientist Atlanta
(3270, 5)


In [12]:
temp1_df = job_db('web+developer', 'San+Jose', 'CA')
temp2_df = job_db('web+developer', 'San+Francisco', 'CA')
temp3_df = job_db('web+developer', 'Seattle', 'WA')
temp4_df = job_db('web+developer', 'Washington', 'DC')
temp5_df = job_db('web+developer', 'New+York', 'NY')
temp6_df = job_db('web+developer', 'Baltimore', 'MD')
temp7_df = job_db('web+developer', 'Boulder', 'CO')
temp8_df = job_db('web+developer', 'San+Diego', 'CA')
temp9_df = job_db('web+developer', 'Denver', 'CO')
temp10_df = job_db('web+developer', 'Huntsville', 'AL')
temp11_df = job_db('web+developer', 'Colorado+Springs', 'CO')
temp12_df = job_db('web+developer', 'Houston', 'TX')
temp13_df = job_db('web+developer', 'Trenton', 'NJ')
temp14_df = job_db('web+developer', 'Dallas', 'TX')
temp15_df = job_db('web+developer', 'Columbus', 'OH')
temp16_df = job_db('web+developer', 'Austin', 'TX')
temp17_df = job_db('web+developer', 'Philadelphia', 'PA')
temp18_df = job_db('web+developer', 'Durham', 'NC')
temp19_df = job_db('web+developer', 'Raleigh', 'NC')
temp20_df = job_db('web+developer', 'Atlanta', 'GA')

web_developer_df = pd.concat([temp1_df, temp2_df, temp3_df, temp4_df, temp5_df,
                              temp6_df, temp7_df, temp8_df, temp9_df, temp10_df,
                              temp11_df, temp12_df, temp13_df, temp14_df, temp15_df,
                              temp16_df, temp17_df, temp18_df, temp19_df, temp20_df], ignore_index=True)
print(web_developer_df.shape)

1404 web+developer San+Jose
1416 web+developer San+Francisco
1423 web+developer Seattle
3429 web+developer Washington
1833 web+developer New+York
880 web+developer Baltimore
440 web+developer Boulder
383 web+developer San+Diego
597 web+developer Denver
79 web+developer Huntsville
68 web+developer Colorado+Springs
369 web+developer Houston
226 web+developer Trenton
780 web+developer Dallas
221 web+developer Columbus
656 web+developer Austin
534 web+developer Philadelphia
376 web+developer Durham
359 web+developer Raleigh
616 web+developer Atlanta
(3640, 5)


In [13]:
temp1_df = job_db('ux+designer', 'San+Jose', 'CA')
temp2_df = job_db('ux+designer', 'San+Francisco', 'CA')
temp3_df = job_db('ux+designer', 'Seattle', 'WA')
temp4_df = job_db('ux+designer', 'Washington', 'DC')
temp5_df = job_db('ux+designer', 'New+York', 'NY')
temp6_df = job_db('ux+designer', 'Baltimore', 'MD')
temp7_df = job_db('ux+designer', 'Boulder', 'CO')
temp8_df = job_db('ux+designer', 'San+Diego', 'CA')
temp9_df = job_db('ux+designer', 'Denver', 'CO')
temp10_df = job_db('ux+designer', 'Huntsville', 'AL')
temp11_df = job_db('ux+designer', 'Colorado+Springs', 'CO')
temp12_df = job_db('ux+designer', 'Houston', 'TX')
temp13_df = job_db('ux+designer', 'Trenton', 'NJ')
temp14_df = job_db('ux+designer', 'Dallas', 'TX')
temp15_df = job_db('ux+designer', 'Columbus', 'OH')
temp16_df = job_db('ux+designer', 'Austin', 'TX')
temp17_df = job_db('ux+designer', 'Philadelphia', 'PA')
temp18_df = job_db('ux+designer', 'Durham', 'NC')
temp19_df = job_db('ux+designer', 'Raleigh', 'NC')
temp20_df = job_db('ux+designer', 'Atlanta', 'GA')

ux_designer_df = pd.concat([temp1_df, temp2_df, temp3_df, temp4_df, temp5_df,
                            temp6_df, temp7_df, temp8_df, temp9_df, temp10_df,
                            temp11_df, temp12_df, temp13_df, temp14_df, temp15_df,
                            temp16_df, temp17_df, temp18_df, temp19_df, temp20_df], ignore_index=True)
print(ux_designer_df.shape)

626 ux+designer San+Jose
876 ux+designer San+Francisco
951 ux+designer Seattle
523 ux+designer Washington
982 ux+designer New+York
89 ux+designer Baltimore
132 ux+designer Boulder
107 ux+designer San+Diego
147 ux+designer Denver
8 ux+designer Huntsville
6 ux+designer Colorado+Springs
82 ux+designer Houston
40 ux+designer Trenton
175 ux+designer Dallas
47 ux+designer Columbus
274 ux+designer Austin
145 ux+designer Philadelphia
80 ux+designer Durham
80 ux+designer Raleigh
252 ux+designer Atlanta
(3018, 5)


In [14]:
temp1_df = job_db('ios_developer', 'San+Jose', 'CA')
temp2_df = job_db('ios_developer', 'San+Francisco', 'CA')
temp3_df = job_db('ios_developer', 'Seattle', 'WA')
temp4_df = job_db('ios_developer', 'Washington', 'DC')
temp5_df = job_db('ios_developer', 'New+York', 'NY')
temp6_df = job_db('ios_developer', 'Baltimore', 'MD')
temp7_df = job_db('ios_developer', 'Boulder', 'CO')
temp8_df = job_db('ios_developer', 'San+Diego', 'CA')
temp9_df = job_db('ios_developer', 'Denver', 'CO')
temp10_df = job_db('ios_developer', 'Huntsville', 'AL')
temp11_df = job_db('ios_developer', 'Colorado+Springs', 'CO')
temp12_df = job_db('ios_developer', 'Houston', 'TX')
temp13_df = job_db('ios_developer', 'Trenton', 'NJ')
temp14_df = job_db('ios_developer', 'Dallas', 'TX')
temp15_df = job_db('ios_developer', 'Columbus', 'OH')
temp16_df = job_db('ios_developer', 'Austin', 'TX')
temp17_df = job_db('ios_developer', 'Philadelphia', 'PA')
temp18_df = job_db('ios_developer', 'Durham', 'NC')
temp19_df = job_db('ios_developer', 'Raleigh', 'NC')
temp20_df = job_db('ios_developer', 'Atlanta', 'GA')

ios_developer_df = pd.concat([temp1_df, temp2_df, temp3_df, temp4_df, temp5_df,
                              temp6_df, temp7_df, temp8_df, temp9_df, temp10_df,
                              temp11_df, temp12_df, temp13_df, temp14_df, temp15_df,
                              temp16_df, temp17_df, temp18_df, temp19_df, temp20_df], ignore_index=True)
print(ios_developer_df.shape)

47 ios_developer San+Jose
43 ios_developer San+Francisco
13 ios_developer Seattle
11 ios_developer Washington
34 ios_developer New+York
5 ios_developer Baltimore
11 ios_developer Boulder
6 ios_developer San+Diego
14 ios_developer Denver
15 ios_developer Huntsville
6 ios_developer Colorado+Springs
6 ios_developer Houston
5 ios_developer Trenton
16 ios_developer Dallas
2 ios_developer Columbus
16 ios_developer Austin
8 ios_developer Philadelphia
5 ios_developer Durham
5 ios_developer Raleigh
13 ios_developer Atlanta
(1764, 5)


In [0]:
data_scientist_df['job'] = 'data scientist'
web_developer_df['job'] = 'web developer'
ux_designer_df['job'] = 'ux designer'
ios_developer_df['job'] = 'ios developer'
indeed_df = pd.concat([data_scientist_df, web_developer_df, ux_designer_df, ios_developer_df], ignore_index=True)
indeed_df = indeed_df.drop_duplicates(keep=False) 
indeed_df = indeed_df.reset_index(drop=True)

In [16]:
indeed_df.head()

,job_title,company,location,salary,description,job
0,Data Scientist,HCL America,None,None,We are looking for a Data Scientistto be part ...,data scientist
1,Data Scientist,Palo Verde Consulting,"Campbell, CA 95008","$150,000 - $210,000 a year","Job Title: Data ScientistLocation: Campbell, C...",data scientist
2,Data Scientist (All Levels) - Santa Clara,LeanTaaS,"Santa Clara, CA 95050",None,Help build technology that saves lives!\n\nWe'...,data scientist
3,Data Scientist,Acubed,"Sunnyvale, CA",None,ADAM\n\nThe Advanced Digital Design and Manufa...,data scientist
4,Data Scientist,Spry Health,"Palo Alto, CA","$100,000 - $135,000 a year",Spry Health’s mission is to build the world’s ...,data scientist


In [17]:
def convert_salary(sal_str, s_flag):
    
    sal_split = re.findall(r'\d+', sal_str.replace(",", ""))
    
    if len(sal_split) == 2:
        low_salary = int(sal_split[0])
        high_salary = int(sal_split[1])
    else:
        low_salary = None
        high_salary = None
    
    if s_flag == 'l':
        salary = low_salary
    else:
        salary = high_salary
    
    return salary

indeed_df['low_salary'] = indeed_df.apply(lambda x: convert_salary(x['salary'], 'l'), axis=1)
indeed_df['high_salary'] = indeed_df.apply(lambda x: convert_salary(x['salary'], 'h'), axis=1)
indeed_df = indeed_df.drop(columns=['salary'])

indeed_df.head()                                      

,job_title,company,location,description,job,low_salary,high_salary
0,Data Scientist,HCL America,None,We are looking for a Data Scientistto be part ...,data scientist,NaN,NaN
1,Data Scientist,Palo Verde Consulting,"Campbell, CA 95008","Job Title: Data ScientistLocation: Campbell, C...",data scientist,150000.0,210000.0
2,Data Scientist (All Levels) - Santa Clara,LeanTaaS,"Santa Clara, CA 95050",Help build technology that saves lives!\n\nWe'...,data scientist,NaN,NaN
3,Data Scientist,Acubed,"Sunnyvale, CA",ADAM\n\nThe Advanced Digital Design and Manufa...,data scientist,NaN,NaN
4,Data Scientist,Spry Health,"Palo Alto, CA",Spry Health’s mission is to build the world’s ...,data scientist,100000.0,135000.0


In [18]:
indeed_df.tail()

,job_title,company,location,description,job,low_salary,high_salary
3695,Senior iOS Developer,InMotion Software,"Austin, TX 78759",About:\nInMotion Software is a full-service de...,ios developer,100000.0,130000.0
3696,Senior iOS Developer,Amherst,"Austin, TX 78704","Senior iOS Developer – Austin, TX\n\nAmherst i...",ios developer,NaN,NaN
3697,"Senior iOS Engineer - Austin, TX",HEB,"Austin, TX 78705","Senior iOS Engineer - Austin, TX\n - (19076378...",ios developer,NaN,NaN
3698,Android/IOS Developer,ficus Inc,"Atlanta, GA",Job SummaryPosition: Android / Ios DeveloperLo...,ios developer,65.0,70.0
3699,iOS Developer,NLB,"Atlanta, GA",1. Job Profile: - IOS Developer Job Location: ...,ios developer,65.0,70.0


In [19]:
def clean_text(text):
    text = text.replace('\\n', ' ')               # remove newline
    text = BeautifulSoup(text, "lxml").get_text() # remove html
    text = text.replace('/', ' ')                 # remove forward slashes
    text = re.sub(r'[^a-zA-Z ^0-9]', '', text)    # letters and numbers only
    text = text.lower()                           # lower case
    text = re.sub(r'(x.[0-9])', '', text)         # remove special characters
    return text

indeed_df['description'] = indeed_df.apply(lambda x: clean_text(x['description']), axis=1)

indeed_df.head()

,job_title,company,location,description,job,low_salary,high_salary
0,Data Scientist,HCL America,None,we are looking for a data scientistto be part ...,data scientist,NaN,NaN
1,Data Scientist,Palo Verde Consulting,"Campbell, CA 95008",job title data scientistlocation campbell ca 9...,data scientist,150000.0,210000.0
2,Data Scientist (All Levels) - Santa Clara,LeanTaaS,"Santa Clara, CA 95050",help build technology that saves liveswere a f...,data scientist,NaN,NaN
3,Data Scientist,Acubed,"Sunnyvale, CA",adamthe advanced digital design and manufactur...,data scientist,NaN,NaN
4,Data Scientist,Spry Health,"Palo Alto, CA",spry healths mission is to build the worlds la...,data scientist,100000.0,135000.0


In [20]:
indeed_df['description'][0]

'we are looking for a data scientistto be part of our tech officewhat are we looking for  masters in data science or phdin data science over 10 years of industry experience and at least 4year experience in machine learning deep learning extensive experience in python or rstudio experience in dl frameworks like tensorflow experiencing in leading mentoring a team of data scientists in ml dl ability to prototype build ml applications using cognitive analytics technologies in close collaboration with solution architecture and software engineering teams prior analytics experience in any one industry or more healthcare automotive aerospace consumer products and manufacturing indicative list experience in solving clients analytics problems and effectively communicating results and methodologies proven ability to rationalize disparate data sources and the ability to intuit the large picture within a dataset knowledge and experience working with hadoop eco system spark and distributed computing

In [21]:
indeed_df['description'][4]

'spry healths mission is to build the worlds largest physiological dataset to map human health understand disease and create new medical knowledge we are innovating in the field of physiological signal processing and are looking for an experienced and motivated researcher to join our talented rd team the research engineer will take big responsibilities on our core science team and break new ground in health monitoring technology through physiological signal processing we are the first to collect continuous physiological signals from thousands of patientsthis is an opportunity to step into a highly innovative healthcare startup and make realworld impact our dataset contains important information that would be a building block in medicine and physiological monitoring discovering new physiological biomarkers that will help us combat disease working with physiological signals from users who are going about their daily life is very challengingwe have made significant progress but are lookin

In [0]:
# convert to sqlite3
conn = sqlite3.connect('techsearch.sqlite3')
curs = conn.cursor()
curs.execute('drop table if exists listings')
indeed_df.to_sql('listings', con=conn)